In [1]:
import json
from sklearn.model_selection import train_test_split
import pandas as pd
import re


In [2]:
with open('data/with_titles_and_abstract.json', 'r') as fin:
    data = json.load(fin)

In [3]:
df = pd.DataFrame(data)

In [4]:
columns_to_drop = [col for col in df.columns if col not in ['title', 'abstract']]

df = df.drop(columns=columns_to_drop, axis=1)

In [5]:
df = df.sample(frac=0.01)

In [7]:
df

,title,abstract
627227,Research on Solution Space of Bipartite Graph ...,Some rigorous results and statistics of the ...
1034356,Smooth perturbations of the functional calculu...,We show for a certain class of operators $A$...
2147065,Stationary Symmetric alpha-Stable Discrete Par...,We establish a connection between the struct...
663445,"Yield, Area and Energy Optimization in Stt-MRA...",Spin Transfer Torque MRAMs are attractive du...
1989391,The averaged null energy condition and differe...,"Recently, Larry Ford and Tom Roman have disc..."
...,...,...
334459,Unification of witnessing initial system-envir...,We show the connection between a witness tha...
1350140,Jiamusi pulsar observations: III. Nulling of 2...,Most of pulsar nulling observations were con...
1647666,Estimation and Inference by Stochastic Optimiz...,"In non-linear estimations, it is common to a..."
1011696,Collaborative Planning for Mixed-Autonomy Lane...,Driving is a social activity: drivers often ...


In [8]:

train_test_ratio = 0.9
train_valid_ratio = 7/9
df_full_train, df_test = train_test_split(df, train_size = train_test_ratio, random_state = 1)
df_train, df_valid = train_test_split(df_full_train, train_size = train_valid_ratio, random_state = 1)

In [11]:
import random

def build_dataset(df, dest_path):
    f = open(dest_path, 'w')
    data = ''
    abstracts = df['abstract'].tolist()
    titles = df['title'].tolist()
    datapoints = [titles[i]+': '+abstracts[i] for i in range(len(titles))]
    for item in datapoints:
        if random.random() < 0.01:
            item = str(item).strip()
            item = re.sub(r"\s", " ", item)
            bos_token = '<BOS>'
            eos_token = '<EOS>'
            data += bos_token + ' ' + item + ' ' + eos_token + '\n'
        
    f.write(data)


In [12]:
build_dataset(df_train, 'data/gpt2_train.txt')
build_dataset(df_valid, 'data/gpt2_valid.txt')
build_dataset(df_test, 'data/gpt2_test.txt')

In [9]:
df.columns

Index(['title', 'abstract'], dtype='object')

In [10]:
df_train.to_json('data/train_small.json')
df_test.to_json('data/test_small.json')
df_valid.to_json('data/valid_small.json')

In [11]:
from datasets import load_dataset

/home/jlunder/papers/.venvs/e/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:


train_set = load_dataset('json', 'data/train_small.json')

Resolving data files: 100%|██████████| 28/28 [00:00<00:00, 40260.72it/s]


InvalidConfigName: Bad characters from black list '<>:/\|?*' found in 'data/train_small.json'. They could create issues when creating a directory for this config on Windows filesystem.

In [25]:
def build_json_dataset(df, path):
    with open(path, 'w') as fout:
        for i in df['title'].keys():
            item = {'title':df['title'][i], 'abstract':df['abstract'][i]}
            fout.write(json.dumps(item, indent=0)+'\n')

In [27]:
build_json_dataset(df_train, 'data/train_small.json')
build_json_dataset(df_valid, 'data/valid_small.json')
build_json_dataset(df_test, 'data/test_small.json')

In [24]:
df_train

,title,abstract
42932,On the kinematic evolution of young local asso...,"Context: Over the last decade, several group..."
562476,The nature and line shapes of charmonium in th...,We explore the nature of sharp resonances wi...
578607,On the Unstable Directions and Lyapunov Expone...,"Despite the invertible setting, Anosov endom..."
1550956,Gravitational spin-orbit dynamics at the fifth...,Accurate waveform models are crucial for gra...
2065529,Holographic Renormalization Group and Fermioni...,We discuss Holographic Renormalization Group...
...,...,...
2154205,Invariant Radon measures on measured laminatio...,Let S be a non-exceptional oriented surface ...
1689502,Efficient Search of the k Shortest Non-Homotop...,An efficient algorithm to solve the $k$ shor...
115392,Dissipative dynamics of a Harmonic Oscillator ...,"Starting from a microscopic theory, we deriv..."
201575,New solutions to the Hurwitz problem on square...,The Hurwitz problem of composition of quadra...


In [14]:
d = df_train.to_dict()

In [19]:
list(d['title'].keys()) == list(d['abstract'].keys())

True

In [17]:
d['abstract']

{42932: '  Context: Over the last decade, several groups of young (mainly low-mass)\nstars have been discovered in the solar neighbourhood (closer than ~100 pc),\nthanks to cross-correlation between X-ray, optical spectroscopy and kinematic\ndata. These young local associations -- including an important fraction whose\nmembers are Hipparcos stars -- offer insights into the star formation process\nin low-density environments, shed light on the substellar domain, and could\nhave played an important role in the recent history of the local interstellar\nmedium. Aims: To study the kinematic evolution of young local associations and\ntheir relation to other young stellar groups and structures in the local\ninterstellar medium, thus casting new light on recent star formation processes\nin the solar neighbourhood. Methods: We compiled the data published in the\nliterature for young local associations. Using a realistic Galactic potential\nwe integrated the orbits for these associations and the